In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
pwd

'C:\\Users\\ashok\\Desktop\\FaceMaskDetection'

In [4]:
import cv2,os

data_path='/content/drive/My Drive/data'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/data'

In [28]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [29]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [30]:
np.save('data',data)
np.save('target',new_target)

In [31]:
data=np.load('data.npy')
target=np.load('target.npy')

In [32]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(100,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [34]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
32/32 [==============================] - 45s 1s/step - loss: 0.7317 - accuracy: 0.5158 - val_loss: 0.6932 - val_accuracy: 0.4743
Epoch 2/20
32/32 [==============================] - 48s 1s/step - loss: 0.6841 - accuracy: 0.5395 - val_loss: 0.6173 - val_accuracy: 0.7233
Epoch 3/20
32/32 [==============================] - 44s 1s/step - loss: 0.5397 - accuracy: 0.7243 - val_loss: 0.4475 - val_accuracy: 0.8419
Epoch 4/20
32/32 [==============================] - 45s 1s/step - loss: 0.3944 - accuracy: 0.8271 - val_loss: 0.3491 - val_accuracy: 0.8656
Epoch 5/20
32/32 [==============================] - 44s 1s/step - loss: 0.2891 - accuracy: 0.8844 - val_loss: 0.2560 - val_accuracy: 0.9170
Epoch 6/20
32/32 [==============================] - 44s 1s/step - loss: 0.2226 - accuracy: 0.9150 - val_loss: 0.1827 - val_accuracy: 0.9328
Epoch 7/20
32/32 [==============================] - 44s 1s/step - loss: 0.1868 - accuracy: 0.9249 - val_loss: 0.1370 - val_accuracy: 0.9486
Epoch 8/20
32/32 [==

In [35]:
print(model.evaluate(test_data,test_target))

5/5 [==============================] - 1s 239ms/step - loss: 0.1477 - accuracy: 0.9220
[0.14765344560146332, 0.9219858050346375]


In [5]:
from keras.models import load_model
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [37]:
%cd '/content'

/content


In [7]:
model = load_model('model-017.model')

face_clsfr=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [10]:
img=cv2.imread('/content/drive/My Drive/download01.jpg')    
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

faces = face_clsfr.detectMultiScale(gray,1.3,5)
if len(faces)>0:
  [x,y,w,h]=faces[0]
  face_img=gray[y:y+w,x:x+w]
  resized=cv2.resize(face_img,(100,100))
  normalized=resized/255.0
  reshaped=np.reshape(normalized,(1,100,100,1))
  result=model.predict(reshaped)

  label=np.argmax(result,axis=1)[0]
        
  cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
  cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
  cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
          
          
  cv2_imshow(img)
else:
  print("Face not found")

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\users\ashok\appdata\local\programs\python\python38\lib\asyncio\selector_events.py", line 120, in _read_from_self
 

In [42]:
!zip -r /content/file.zip /content/model-017.model

  adding: content/model-017.model/ (stored 0%)
  adding: content/model-017.model/assets/ (stored 0%)
  adding: content/model-017.model/saved_model.pb (deflated 90%)
  adding: content/model-017.model/variables/ (stored 0%)
  adding: content/model-017.model/variables/variables.index (deflated 64%)
  adding: content/model-017.model/variables/variables.data-00000-of-00001 (deflated 5%)


In [43]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>